In [36]:
import numpy as np
import cv2
import os
import sys
from PIL import Image
import png
from matplotlib import pyplot as plt
import matplotlib.lines as mlines
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter

In [2]:
def plotImage(img, edges = None):
    # plot the image with edges or only image
    if not edges:
        fig=plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
        plt.gray()
        plt.imshow(img)
    else:
        fig=plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
        plt.gray()
        plt.imshow(img)
        plt.plot()
        l = mlines.Line2D([xmin,xmax], [ymin,ymax])

In [3]:
def plotProfile(profile):
    fig=plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
    plt.plot(range(len(profile)), profile)

In [4]:
def imgFiltering(img):
    median = cv2.medianBlur(img, 3)
    return median

In [5]:
def smooth(x, window_len=11, window='hanning'):
    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    print(len(s))
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y

In [6]:
def removeCloseGrad(gradients):
    # after thresholding the gradient of the profile, same large edge may create some consecutive large gradients
    nonConsecutiveEdges = [gradients[0]]
    counter = 1
    for gradient in gradients:
        if gradient - nonConsecutiveEdges[-1] == 0:
            pass
        elif gradient - nonConsecutiveEdges[-1] == counter:
            counter += 1
        else:
            nonConsecutiveEdges.append(gradient)
            counter = 1
    # the first one is fake edge created by color filling
    return nonConsecutiveEdges[1:]

In [7]:
def filterAccordingToWidth(gradients):
    # WARNING!!
    # this is a cheating trick, manually measured the tow width -> around 140 pxls
    towWidth = 140
    tows = [gradients[0]]
    for gradient in gradients:
        if (gradient - tows[0])%towWidth < 20 or (gradient - tows[0])%towWidth > 120:
            tows.append(gradient)
    return tows[1:]

In [8]:
# find consecutive zeros
def zero_runs(a):
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges

In [9]:
# processing steps from Profactor, first interpolate the black dots
def filterRow(row):
    zeroRanges = zero_runs(row)
    if zeroRanges.size != 0:
        for zeroRange in zeroRanges:
            if zeroRange[0] == 0 and zeroRange[1] != len(row):
                row[zeroRange[0] : zeroRange[1]].fill(row[zeroRange[1]])
            elif zeroRange[0] != 0 and zeroRange[1] == len(row):
                row[zeroRange[0] : zeroRange[1]].fill(row[zeroRange[0]-1])
            elif zeroRange[0] != 0 and zeroRange[1] != len(row):
                x = [zeroRange[0] - 1, zeroRange[1]]
                y = [row[zeroRange[0] - 1], row[zeroRange[1]]]
                row[zeroRange[0] : zeroRange[1]] = np.interp(range(zeroRange[0], zeroRange[1]), x, y)
    else:
        return row
    return row

In [10]:
def printStatus(fileName, img):
#     print('The file name is', fileName, \
#              '\nThe image size', img.shape, \
#              '\nThe whole image mean value:', np.mean(img), \
#              '\nThe maximum value:', np.amax(img), \
#              '\nThe minimum value:', np.amin(img), \
#              '\n\nThe image left part, mean value:', np.mean(img[:, :1000]), \
#              '\nThe image left part, max value:', np.amax(img[:, :1000]), \
#              '\nThe image left part, min value:', np.amin(img[:, :1000]), \
#              '\n\nThe image right part, mean value:', np.mean(img[:, 1000:]), \
#              '\nThe image right part, max value:', np.amax(img[:, 1000:]), \
#              '\nThe image right part, min value:', np.amin(img[:, 1000:]), \
#              '\n-------------------------------------------------------\n')
    print('The file name is', fileName, \
             '\nThe image size', img.shape, \
             '\nThe whole image mean value:', np.mean(img), \
             '\nThe maximum value:', np.amax(img), \
             '\nThe minimum value:', np.amin(img), \
             '\n-------------------------------------------------------\n')

In [11]:
def normalizeImg(img):
    """
    Normalize the image to [0, 255], may not be necessary
    """
    normalizedImg = img - np.amin(img)
    normalizedImg = np.multiply((255.0/normalizedImg.max()), normalizedImg)
    normalizedImg = normalizedImg.astype(int)
    return normalizedImg

In [12]:
def saveImg(fileName, targetDir, img):
    nameDir = targetDir + fileName
    
    # Use pypng to write img as a grayscale PNG.
    with open(nameDir, 'wb') as f:
        writer = png.Writer(width=img.shape[1], height=img.shape[0], bitdepth=16, greyscale=True)
        gray2list = img.tolist()
        writer.write(f, gray2list)

In [13]:
def fitProfileAndPlot(profile):
    # define a function to be fitted
    def func(x, a, b, c):
        return np.square(x)*a + x*b + c 
    
    # plot the fitted function on the profile
    plotProfile(profile)
    popt, pcov = curve_fit(func, range(len(profile)), profile)
    plt.plot(range(len(profile)), func(range(len(profile)), *popt), \
             'g--', label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))

In [14]:
def fitProfileAndNormalize(profile):
    # define a function to be fitted
    def func(x, a, b, c):
        return np.square(x)*a + x*b + c 
    
    # normalize the profile and avoid negative value
    popt, pcov = curve_fit(func, range(len(profile)), profile)
    [a, b, c] = popt
    minVal = np.amin(profile - func(range(len(profile)), a, b, c)) 
    return profile - func(range(len(profile)), a, b, c) - minVal

In [15]:
def fitProfile(profile):
    # define a function to be fitted
    def func(x, a, b, c):
        return np.square(x)*a + x*b + c 
    
    # normalize the profile and avoid negative value
    popt, pcov = curve_fit(func, range(len(profile)), profile)
    return popt

In [16]:
def interpolationRows(paramsTop, paramsBottom, wTop, wBottom):
    """
    Interpolation of two fixed fitting curves to get a fitting curve at any row in a frame.
    Return the interpolated parameters of a curve.
    """
    return paramsTop*wTop + paramsBottom*wBottom

In [17]:
def colorNormalize(img):
    # scaling factor
    maxPxlValue = img.max()
    minPxlValue = img.min()
    if maxPxlValue == minPxlValue:
        return None
    
    alpha = 255/(maxPxlValue-minPxlValue)

    # shifting factor
    beta = -min(minPxlValue * alpha, maxPxlValue * alpha)
    normalizedImg = np.float32(img) * alpha + beta
    normalizedImg = np.uint8(normalizedImg)
    im_color = cv2.applyColorMap(normalizedImg, cv2.COLORMAP_BONE)
    return normalizedImg

In [53]:
sourceFolder = '/Users/zeekile/Desktop/Infy/nlr_data/20200422_Noon_range/'
# sourceFolder = '/Users/zeekile/Downloads/'
# sourceFolder = '/Users/zeekile/Desktop/Infy/nlr_data/goodRange/'

for fileName in os.listdir(sourceFolder): 
    if fileName.endswith(".png"):
        print(fileName)
        filePath = os.path.join(sourceFolder, fileName)
        img = (cv2.imread(filePath, -1)).astype('int32')
        imgOriginalCopy = np.copy(img)
        imgFilled = np.copy(img)
        # printStatus(fileName, img)
        
        # fill the black holes
        # plotImage(imgOriginalCopy)
        for rowId in range(len(imgFilled)):
            imgFilled[rowId] = filterRow(imgFilled[rowId])
            # imgFilled[rowId] = fitProfileAndNormalize(imgFilled[rowId])
        # plotImage(imgFilled)
        
        # fit two selected profiles, and substract the frame with interpolation
        paramsTop = fitProfile(imgFilled[0])
        paramsMiddle = fitProfile(imgFilled[int(len(imgFilled)/2)])
        paramsBottom = fitProfile(imgFilled[-1])
        x = np.array(range(len(imgFilled[0])))
        for rowId in range(len(imgFilled)):
            if rowId <= int(len(imgFilled)/2):
                wBottom = rowId / int(len(imgFilled)/2)
                wTop = 1 - wBottom
                [a,b,c] = interpolationRows(paramsTop, paramsMiddle, wTop, wBottom)
                imgFilled[rowId] = imgFilled[rowId] - (np.square(x)*a + x*b + c)
            else:
                wBottom = (rowId - int(len(imgFilled)/2)) / (len(imgFilled) - int(len(imgFilled)/2))
                wTop = 1 - wBottom
                [a,b,c] = interpolationRows(paramsMiddle, paramsBottom, wTop, wBottom)
                imgFilled[rowId] = imgFilled[rowId] - (np.square(x)*a + x*b + c)
        minVal = np.amin(imgFilled) 
        imgFilled = imgFilled - minVal
        
        # prune some pure black images
        if ((imgFilled[np.nonzero(imgFilled)]).shape)[0] == 0:
            continue
        minval = np.min(imgFilled[np.nonzero(imgFilled)])
        normalized = (((imgFilled - minval)/(np.max(imgFilled) - minval)) * 255).astype('int')
        # plotImage(normalized)
        
        try:
            saveImg(fileName, '/Volumes/ZEEKANDISK/nlr/normalized/20200422_Noon_range/', normalized)
        except:
            print("[0]Unexpected error from file:", fileName, sys.exc_info()[0])
            # raise
        
        try:
            normalized = np.uint8(normalized)
            im_color = cv2.applyColorMap(normalized, cv2.COLORMAP_BONE)
            cv2.imwrite('/Volumes/ZEEKANDISK/nlr/colored/20200422_Noon_range/'+fileName, im_color)
        except:
            print("[1]Unexpected error from file:", fileName, sys.exc_info()[0])
            # raise
        # printStatus(fileName, normalized)

        # create a mean vertically
        # means = np.mean(imgFilled, axis=0)
        # smoothed = savgol_filter(means, 7, 5)
        # fitProfileAndPlot(imgFilled[100])

        # gradientArr = np.gradient(smoothed)
        # largeGradIndex = removeCloseGrad((np.nonzero(gradientArr>50))[0])
        # towXs = filterAccordingToWidth(largeGradIndex)
        # print(towXs)
        # plotProfile(gradientArr)
        # plotProfile(np.diff(smoothed, n=3))
        # plt.show()
        
        # 
#     break

buffer_2020-04-22-115329.911259365.json_range.png
[0]Unexpected error from file: buffer_2020-04-22-115329.911259365.json_range.png <class 'struct.error'>
buffer_2020-04-22-091256.870280297.json_range.png
[0]Unexpected error from file: buffer_2020-04-22-091256.870280297.json_range.png <class 'struct.error'>
buffer_2020-04-22-091309.378402803.json_range.png
[0]Unexpected error from file: buffer_2020-04-22-091309.378402803.json_range.png <class 'struct.error'>
buffer_2020-04-22-120519.855967120.json_range.png
buffer_2020-04-22-120402.973711040.json_range.png
buffer_2020-04-22-115910.675378034.json_range.png
[0]Unexpected error from file: buffer_2020-04-22-115910.675378034.json_range.png <class 'struct.error'>
buffer_2020-04-22-091332.766016200.json_range.png
buffer_2020-04-22-120116.540079511.json_range.png
buffer_2020-04-22-100127.730281882.json_range.png
buffer_2020-04-22-092249.454332339.json_range.png
buffer_2020-04-22-120337.887836951.json_range.png
buffer_2020-04-22-115917.45974284.

buffer_2020-04-22-092016.10152304.json_range.png
buffer_2020-04-22-115621.739768450.json_range.png
buffer_2020-04-22-115928.878714829.json_range.png
buffer_2020-04-22-120307.626734267.json_range.png
buffer_2020-04-22-120329.199261159.json_range.png
buffer_2020-04-22-092223.539805208.json_range.png
buffer_2020-04-22-115816.217308563.json_range.png
buffer_2020-04-22-100444.26513694.json_range.png
buffer_2020-04-22-092605.46443774.json_range.png
buffer_2020-04-22-100825.449848557.json_range.png
buffer_2020-04-22-091649.161438088.json_range.png
buffer_2020-04-22-120106.984218817.json_range.png
buffer_2020-04-22-115822.587906310.json_range.png
buffer_2020-04-22-095933.701597088.json_range.png
[0]Unexpected error from file: buffer_2020-04-22-095933.701597088.json_range.png <class 'struct.error'>
buffer_2020-04-22-092133.372570943.json_range.png
buffer_2020-04-22-092243.83794306.json_range.png
buffer_2020-04-22-115737.290858484.json_range.png
buffer_2020-04-22-115708.859604743.json_range.png


buffer_2020-04-22-115501.14461005.json_range.png
buffer_2020-04-22-115658.676513194.json_range.png
buffer_2020-04-22-092028.934013834.json_range.png
buffer_2020-04-22-091624.854053519.json_range.png
buffer_2020-04-22-100251.824614491.json_range.png
buffer_2020-04-22-120406.159286664.json_range.png
buffer_2020-04-22-120326.13856425.json_range.png
buffer_2020-04-22-091247.314180023.json_range.png
buffer_2020-04-22-100235.957077282.json_range.png
buffer_2020-04-22-115305.93561014.json_range.png
buffer_2020-04-22-100015.756160397.json_range.png
[0]Unexpected error from file: buffer_2020-04-22-100015.756160397.json_range.png <class 'struct.error'>
buffer_2020-04-22-115800.751090726.json_range.png
buffer_2020-04-22-100034.604595497.json_range.png
buffer_2020-04-22-091604.457445508.json_range.png
buffer_2020-04-22-120600.736232522.json_range.png
buffer_2020-04-22-092110.674399079.json_range.png
buffer_2020-04-22-092856.65817258.json_range.png
buffer_2020-04-22-120459.719777600.json_range.png


[0]Unexpected error from file: buffer_2020-04-22-092252.639793300.json_range.png <class 'struct.error'>
buffer_2020-04-22-115750.689473660.json_range.png
buffer_2020-04-22-095046.801414349.json_range.png
buffer_2020-04-22-100328.427439873.json_range.png
buffer_2020-04-22-115721.600735717.json_range.png
[0]Unexpected error from file: buffer_2020-04-22-115721.600735717.json_range.png <class 'struct.error'>
buffer_2020-04-22-120239.825400819.json_range.png
[0]Unexpected error from file: buffer_2020-04-22-120239.825400819.json_range.png <class 'struct.error'>
buffer_2020-04-22-091537.988906777.json_range.png
buffer_2020-04-22-115425.589965285.json_range.png
buffer_2020-04-22-093043.355865894.json_range.png
buffer_2020-04-22-100420.510619166.json_range.png
buffer_2020-04-22-120251.820150744.json_range.png
buffer_2020-04-22-100449.278566841.json_range.png
buffer_2020-04-22-120545.140191943.json_range.png
buffer_2020-04-22-091723.764849331.json_range.png
buffer_2020-04-22-092746.2503947.json_